In [129]:
import sympy as sp
from sympy.abc import a, b, t, r
phi1, phi2 = sp.symbols("phi_1 phi_2")

In [16]:
m = a-b*sp.cos(phi2)
n = b-a*sp.cos(phi2)

In [19]:
x = m*sp.sin(phi1)
y = m*sp.cos(phi1)
z = b*sp.sin(phi2)

param = (x, y, z)

In [139]:
from itertools import product
from functools import reduce

def d2(expr, syms=(phi1, phi2)):
    n = len(syms)
    combs = product(*(n*[syms]))
    ijth_term = lambda i, j: expr.diff(i)*expr.diff(j)
    return sp.Array([reduce(ijth_term, comb) for comb in combs]).reshape(2, 2)

def line_element(param, syms=(phi1, phi2)):
    n = len(syms)
    componenets = sp.Array([d2(c) for c in param])
    return reduce(sp.Array.__add__, componenets).simplify()

le = line_element(param)

In [213]:
def anstaz(element, extra=[t, r]):
    m = len(extra)
    n = element.shape[0]
    tot = m + n
    
    Phi = sp.Function("Phi")(r, t)
    Psi = sp.Function("Psi")(r, t)
    
    g = sp.zeros(tot)
    
    g[0, 0] = -sp.exp(Phi)
    g[1, 1] = sp.exp(Psi)
    
    g[2:, 2:] = element
    return g


def christoffel(g, syms=(t, r, phi1, phi2)):
    Gamma = np.zeros((4, 4, 4)).astype('object')#sp.Array(np.array(
    g_inv = g.inv()
    for rho in trange(4):
        for mu in range(4):
            for nu in range(4):
                ch = lambda gamma: 0.5*g.inv()[rho, gamma]*(g[nu, gamma].diff(syms[mu]) + g[mu, gamma].diff(syms[nu]) - g[mu, nu].diff(syms[gamma]))
                #print(f"rho={rho}, mu={mu}, nu={nu}",0.5*sum([ch(gamma).simplify() for gamma in range(4)]))
                Gamma[rho, mu, nu] = sum([ch(gamma).simplify() for gamma in range(4)])
    return sp.Array(Gamma)
                
g = anstaz(le)
G = christoffel(g)

100%|██████████| 4/4 [00:43<00:00, 10.96s/it]


In [212]:
G

[[[1.5*Derivative(Phi(r, t), t), 0.5*Derivative(Phi(r, t), r), 0, 0], [0.5*Derivative(Phi(r, t), r), -0.5*exp(-Phi(r, t) + Psi(r, t))*Derivative(Psi(r, t), t), 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[-0.5*exp(Phi(r, t) - Psi(r, t))*Derivative(Phi(r, t), r), 0.5*Derivative(Psi(r, t), t), 0, 0], [0.5*Derivative(Psi(r, t), t), 1.5*Derivative(Psi(r, t), r), 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1.0*b*sin(phi_2)/(a - b*cos(phi_2))], [0, 0, 1.0*b*sin(phi_2)/(a - b*cos(phi_2)), 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1.0*(a - b*cos(phi_2))*sin(phi_2)/b, 0], [0, 0, 0, 0]]]

In [214]:
G

[[[0.5*Derivative(Phi(r, t), t), 0.5*Derivative(Phi(r, t), r), 0, 0], [0.5*Derivative(Phi(r, t), r), 0.5*exp(-Phi(r, t) + Psi(r, t))*Derivative(Psi(r, t), t), 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0.5*exp(Phi(r, t) - Psi(r, t))*Derivative(Phi(r, t), r), 0.5*Derivative(Psi(r, t), t), 0, 0], [0.5*Derivative(Psi(r, t), t), 0.5*Derivative(Psi(r, t), r), 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1.0*b*sin(phi_2)/(a - b*cos(phi_2))], [0, 0, 1.0*b*sin(phi_2)/(a - b*cos(phi_2)), 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1.0*(-a + b*cos(phi_2))*sin(phi_2)/b, 0], [0, 0, 0, 0]]]

In [217]:
g

Matrix([
[-exp(Phi(r, t)),              0,                     0,    0],
[              0, exp(Psi(r, t)),                     0,    0],
[              0,              0, (a - b*cos(phi_2))**2,    0],
[              0,              0,                     0, b**2]])

In [231]:
def Ricci_tensor(G, syms=[t, r, phi1, phi2]):
    R = sp.zeros(4)
    
    for j in range(4):
        for k in range(4):
            first = sp.Array([G[i, j, k].diff(syms[i]) for i in range(4)])
            second = sp.Array([-G[i, k, i].diff(syms[j]) for i in range(4)])
            third = sp.Array([G[i, i, p]*G[p, j, k] for i, p in product(range(4), range(4))])
            fourth = sp.Array([-G[i, j, p]*G[p, i, k] for i, p in product(range(4), range(4))])
            
            #print(first, second, third, fourth)
            
            R[j, k] = np.sum(first) + np.sum(second) + np.sum(third) + np.sum(fourth)
            
    return sp.Array(R).simplify()

def Ricci_scalar(Rt, syms=[t, r, phi1, phi2]):
    pass

4*Ricci_tensor(G).simplify()

[[exp(Phi(r, t) - Psi(r, t))*Derivative(Phi(r, t), r)**2 - 1.0*exp(Phi(r, t) - Psi(r, t))*Derivative(Phi(r, t), r)*Derivative(Psi(r, t), r) + 2.0*exp(Phi(r, t) - Psi(r, t))*Derivative(Phi(r, t), (r, 2)) + Derivative(Phi(r, t), t)*Derivative(Psi(r, t), t) - 1.0*Derivative(Psi(r, t), t)**2 - 2.0*Derivative(Psi(r, t), (t, 2)), 0, 0, 0], [0, -1.0*exp(-Phi(r, t) + Psi(r, t))*Derivative(Phi(r, t), t)*Derivative(Psi(r, t), t) + exp(-Phi(r, t) + Psi(r, t))*Derivative(Psi(r, t), t)**2 + 2.0*exp(-Phi(r, t) + Psi(r, t))*Derivative(Psi(r, t), (t, 2)) - 1.0*Derivative(Phi(r, t), r)**2 + Derivative(Phi(r, t), r)*Derivative(Psi(r, t), r) - 2.0*Derivative(Phi(r, t), (r, 2)), 0, 0], [0, 0, 4.0*(-a + b*cos(phi_2))*cos(phi_2)/b, 0], [0, 0, 0, -4.0*b*cos(phi_2)/(a - b*cos(phi_2))]]

In [188]:
X = np.zeros((4, 4, 4))

In [199]:
A = np.array([0, 1, 2])

B = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])

In [200]:
np.newaxis

A[:, np.newaxis] * B

array([[ 0,  0,  0,  0],
       [ 4,  5,  6,  7],
       [16, 18, 20, 22]])

In [201]:
A[:, np.newaxis]

array([[0],
       [1],
       [2]])

In [210]:
rho, mu, nu = 1, 1, 1

syms=(t, r, phi1, phi2)

ch = lambda gamma: g.inv()[rho, gamma]*(g[nu, gamma].diff(syms[gamma]) + g[mu, gamma].diff(syms[nu]) + g[mu, nu].diff(syms[gamma]))
[ch(gamma) for gamma in range(4)]

[0, 3*Derivative(Psi(r, t), r), 0, 0]

In [ ]:
g.inv()[1, 1]*(g[1, 1] + )

In [215]:
G

[[[0.5*Derivative(Phi(r, t), t), 0.5*Derivative(Phi(r, t), r), 0, 0], [0.5*Derivative(Phi(r, t), r), 0.5*exp(-Phi(r, t) + Psi(r, t))*Derivative(Psi(r, t), t), 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0.5*exp(Phi(r, t) - Psi(r, t))*Derivative(Phi(r, t), r), 0.5*Derivative(Psi(r, t), t), 0, 0], [0.5*Derivative(Psi(r, t), t), 0.5*Derivative(Psi(r, t), r), 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1.0*b*sin(phi_2)/(a - b*cos(phi_2))], [0, 0, 1.0*b*sin(phi_2)/(a - b*cos(phi_2)), 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1.0*(-a + b*cos(phi_2))*sin(phi_2)/b, 0], [0, 0, 0, 0]]]

In [228]:
from einsteinpy.symbolic.christoffel import ChristoffelSymbols
from einsteinpy.symbolic import MetricTensor

Phi = sp.Function("Phi")(r, t)
Psi = sp.Function("Psi")(r, t)

ein_g = MetricTensor(np.diag([-sp.exp(Phi), sp.exp(Psi), m**2 , b**2]), syms=(t, r, phi1, phi2))
ein_g.tensor()
ChristoffelSymbols.from_metric(ein_g).tensor().simplify()

[[[Derivative(Phi(r, t), t)/2, Derivative(Phi(r, t), r)/2, 0, 0], [Derivative(Phi(r, t), r)/2, exp(-Phi(r, t) + Psi(r, t))*Derivative(Psi(r, t), t)/2, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[exp(Phi(r, t) - Psi(r, t))*Derivative(Phi(r, t), r)/2, Derivative(Psi(r, t), t)/2, 0, 0], [Derivative(Psi(r, t), t)/2, Derivative(Psi(r, t), r)/2, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, b*sin(phi_2)/(a - b*cos(phi_2))], [0, 0, b*sin(phi_2)/(a - b*cos(phi_2)), 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, (-a + b*cos(phi_2))*sin(phi_2)/b, 0], [0, 0, 0, 0]]]